In [2]:
!gsutil cp gs://exploration-novo/Yolov8/SkinProblem.zip .

Copying gs://exploration-novo/Yolov8/SkinProblem.zip...
/ [1 files][275.6 MiB/275.6 MiB]   33.4 MiB/s                                   
Operation completed over 1 objects/275.6 MiB.                                    


In [3]:
!unzip SkinProblem.zip -d .

Archive:  SkinProblem.zip
   creating: ./SkinProblem/
  inflating: ./SkinProblem/data.yaml  
  inflating: ./SkinProblem/README.dataset.txt  
  inflating: ./SkinProblem/README.roboflow.txt  
   creating: ./SkinProblem/test/
   creating: ./SkinProblem/test/images/
  inflating: ./SkinProblem/test/images/00721_png_jpg.rf.731982a8d76d2d37b34f076c12fa4e29.jpg  
  inflating: ./SkinProblem/test/images/00765_png_jpg.rf.6c5032e80a9a147582f060fa35e6d0ed.jpg  
  inflating: ./SkinProblem/test/images/01321_png_jpg.rf.aa8accacee94e698239de1b268d93860.jpg  
  inflating: ./SkinProblem/test/images/01587_png_jpg.rf.f0120cb4b86442a4678c9de828480143.jpg  
  inflating: ./SkinProblem/test/images/01629_png_jpg.rf.b43f8a7b8de361faf260fcec5eac7c32.jpg  
  inflating: ./SkinProblem/test/images/01689_png_jpg.rf.44fa873183491cc430ee3eb2215c6ed9.jpg  
  inflating: ./SkinProblem/test/images/02093_png_jpg.rf.8472b4896784ccdc651dc324d49a768c.jpg  
  inflating: ./SkinProblem/test/images/03035_png_jpg.rf.a9c67fd0d78880ea

In [5]:
# Install dependencies
!pip install ultralytics --upgrade

# Assuming you upload your SkinProblem dataset zip to the Notebook environment
# Unzip dataset
# !unzip -o SkinProblem.zip -d /workspace/

# Check files
# !ls /workspace/SkinProblem/

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.6/783.6 kB 39.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 MB 156.0 MB/s eta 0:00:0000:01
  Attempting uninstall: ultralytics-thop━━━━━━━━━━━━━━━━━━━━━━━━━━ 1/4 [polars]
    Found existing installation: ultralytics-thop 2.0.14━━━━━━ 1/4 [polars]
    Uninstalling ultralytics-thop-2.0.14:━━━━━━━━━━━━━━━━━━━━━ 1/4 [polars]
      Successfully uninstalled ultralytics-thop-2.0.14━━━━━━━━ 1/4 [polars]
  Attempting uninstall: ultralytics0m╺━━━━━━━━━━━━━━━━━━━ 2/4 [ultralytics-thop]
    Found existing installation: ultralytics 8.3.169━━━━━━━━━━ 2/4 [ultralytics-thop]
    Uninstalling ultralytics-8.3.169:m━━━━━━━━━━━━━━━━━━━ 2/4 [ultralytics-thop]
      Successfully uninstalled ultralytics-8.3.169━━━━━━━━━━━━ 2/4 [ultralytics-thop]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4/4 [ultralytics] [ultralytics]]


In [ ]:
# Define dataset paths
import os

# Updated path for your Vertex AI instance
data_path = "/home/jupyter/KartikYolo/SkinProblem"  # root dir

img_dir_train = os.path.join(data_path, "train", "images")
img_dir_valid = os.path.join(data_path, "valid", "images")
lbl_dir_train = os.path.join(data_path, "train", "labels")
lbl_dir_valid = os.path.join(data_path, "valid", "labels")

print("Train images/labels: ", len(os.listdir(img_dir_train)), "/", len(os.listdir(lbl_dir_train)))
print("Valid images/labels: ", len(os.listdir(img_dir_valid)), "/", len(os.listdir(lbl_dir_valid)))

# Train model
from ultralytics import YOLO

model = YOLO("yolov8m-seg.pt")

results = model.train(
    data=os.path.join(data_path, "data.yaml"),  # dataset config
    epochs=100,                # longer = better convergence
    imgsz=640,                 # standard for segmentation
    batch=16,                  # you can go up to 32 if GPU memory > 16GB
    workers=4,                 # parallel data loading
    optimizer="AdamW",         # stable and modern optimizer
    lr0=0.001,                 # good starting LR for AdamW
    lrf=0.01,                  # final LR fraction (for cosine scheduler)
    cos_lr=True,               # smooth cosine learning rate decay
    momentum=0.937,            # default, keep as is
    weight_decay=0.0005,       # regularization
    dropout=0.2,               # improves generalization
    augment=True,              # enable built-in augmentations
    mosaic=1.0,                # mix images for robustness
    mixup=0.2,                 # mixup augmentation for variety
    cache=True,                # cache dataset in RAM for speed
    pretrained=True,           # use pretrained backbone
    patience=20,               # early stop patience
    device=0,                  # use GPU
    verbose=True,
    name="skin-seg-optimized",
    project="/home/jupyter/KartikYolo/runs"  # Updated to your working directory
)

# Save weights to SkinProblem folder
import shutil
shutil.copy("/home/jupyter/KartikYolo/runs/skin-seg-optimized/weights/best.pt", 
            "/home/jupyter/KartikYolo/SkinProblem/")
shutil.copy("/home/jupyter/KartikYolo/runs/skin-seg-optimized/weights/last.pt", 
            "/home/jupyter/KartikYolo/SkinProblem/")

# Load trained model
model = YOLO("/home/jupyter/KartikYolo/SkinProblem/best.pt")

# Validate
model.val(data=os.path.join(data_path, "data.yaml"), split="val")
model.val(data=os.path.join(data_path, "data.yaml"), split="test")

Train images/labels:  5067 / 5067
Valid images/labels:  449 / 449
New https://pypi.org/project/ultralytics/8.3.225 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.224 🚀 Python-3.10.18 torch-2.7.1+cu126 CUDA:0 (NVIDIA A100-SXM4-40GB, 40326MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/jupyter/KartikYolo/SkinProblem/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.2, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.2, mode=train, model=yolov8m

ultralytics.utils.metrics.SegmentMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f22dbb548e0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)', 'Precision-Recall(M)', 'F1-Confidence(M)', 'Precision-Confidence(M)', 'Recall-Confidence(M)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.04

In [3]:
!zip -r  SkinProblem_runs2.zip runs

  adding: runs/ (stored 0%)
  adding: runs/skin-seg-optimized/ (stored 0%)
  adding: runs/skin-seg-optimized/val_batch1_labels.jpg (deflated 6%)
  adding: runs/skin-seg-optimized/labels.jpg (deflated 31%)
  adding: runs/skin-seg-optimized/val_batch0_labels.jpg (deflated 6%)
  adding: runs/skin-seg-optimized/MaskP_curve.png (deflated 9%)
  adding: runs/skin-seg-optimized/results.csv (deflated 61%)
  adding: runs/skin-seg-optimized/train_batch28531.jpg (deflated 6%)
  adding: runs/skin-seg-optimized/results.png (deflated 8%)
  adding: runs/skin-seg-optimized/MaskPR_curve.png (deflated 10%)
  adding: runs/skin-seg-optimized/train_batch28530.jpg (deflated 7%)
  adding: runs/skin-seg-optimized/confusion_matrix_normalized.png (deflated 27%)
  adding: runs/skin-seg-optimized/train_batch1.jpg (deflated 2%)
  adding: runs/skin-seg-optimized/train_batch0.jpg (deflated 1%)
  adding: runs/skin-seg-optimized/val_batch2_labels.jpg (deflated 7%)
  adding: runs/skin-seg-optimized/val_batch0_pred.jpg (